In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


# Bronze path
bronze_df =spark.read.option("multiline", "true").json("Files/Covid_bronze.json")


display(bronze_df)




StatementMeta(, 962aa11b-62b5-49b2-a788-dc5297c4f47a, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1cd5f65a-3ee4-4ee6-adf1-edb393113c12)

In [2]:
# Flatten nested fields from 'countryInfo'
silver_df = (
    bronze_df
    .withColumn("country_id", F.col("countryInfo._id"))
    .withColumn("iso2", F.col("countryInfo.iso2"))
    .withColumn("iso3", F.col("countryInfo.iso3"))
    .withColumn("latitude", F.col("countryInfo.lat"))
    .withColumn("longitude", F.col("countryInfo.long"))
    .withColumn("flag_url", F.col("countryInfo.flag"))
    .drop("countryInfo")
)

display(silver_df)
#silver_df.printSchema()


StatementMeta(, 962aa11b-62b5-49b2-a788-dc5297c4f47a, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dfa1832e-bf13-41c8-b990-2c81e7c0eb5d)

In [3]:
silver_df = (
    silver_df
    .withColumn("death_rate_pct", F.round((F.col("deaths") / F.col("cases")) * 100, 2))
    .withColumn("recovery_rate_pct", F.round((F.col("recovered") / F.col("cases")) * 100, 2))
    .withColumn("active_rate_pct", F.round((F.col("active") / F.col("cases")) * 100, 2))
    .withColumn("tests_per_thousand", F.round(F.col("tests") / F.col("population") * 1000, 2))
    .withColumn("cases_per_million", F.round(F.col("casesPerOneMillion"), 2))
)

# Drop rows missing critical values
silver_df = silver_df.na.drop(subset=["cases", "deaths", "recovered", "population"])

display(silver_df)

StatementMeta(, 962aa11b-62b5-49b2-a788-dc5297c4f47a, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 734635e2-37cf-4a07-8b1e-92f06b3a6365)

In [4]:
# Save as Silver Table
silver_table = "Covid_Silver"
silver_df.write.mode("overwrite").saveAsTable(silver_table)

print(f"✅ Silver table '{silver_table}' created successfully")


StatementMeta(, 962aa11b-62b5-49b2-a788-dc5297c4f47a, 6, Finished, Available, Finished)

✅ Silver table 'Covid_Silver' created successfully
